In [3]:
import cv2
import numpy as np
import time

# Load YOLO with CUDA support (if available)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load four videos with your specified file names
caps = [cv2.VideoCapture(f'traffic_vid{i}.mp4') for i in ['', '_2', '_3', '_4']]

# Record start time
start_time = time.time()

while all(cap.isOpened() for cap in caps):
    frames = []
    for cap in caps:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    if len(frames) < 4:
        break

    # Check if 10 seconds have passed
    if time.time() - start_time > 10:
        break

    # Resize frames for a 2x2 grid layout (each frame size: 683x384 for a 1366x768 output)
    resized_frames = [cv2.resize(frame, (683, 384)) for frame in frames]

    # Create a black canvas for the output
    output_frame = np.zeros((768, 1366, 3), dtype=np.uint8)

    # Assign each resized frame to its respective quadrant
    output_frame[:384, :683] = resized_frames[0]  # Top-left
    output_frame[:384, 683:] = resized_frames[1]  # Top-right
    output_frame[384:, :683] = resized_frames[2]  # Bottom-left
    output_frame[384:, 683:] = resized_frames[3]  # Bottom-right

    # Process each quadrant independently
    for i, frame in enumerate(resized_frames):
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                detection = detection.reshape(-1, 85)
                for obj in detection:
                    scores = obj[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(obj[0] * width)
                        center_y = int(obj[1] * height)
                        w = int(obj[2] * width)
                        h = int(obj[3] * height)

                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

        if indexes is not None:
            for j in indexes.flatten():
                x, y, w, h = boxes[j]
                label = str(classes[class_ids[j]])
                color = (0, 255, 0) if label == "car" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Place the processed frame back into the output frame
        if i == 0:
            output_frame[:384, :683] = frame  # Top-left
        elif i == 1:
            output_frame[:384, 683:] = frame  # Top-right
        elif i == 2:
            output_frame[384:, :683] = frame  # Bottom-left
        else:
            output_frame[384:, 683:] = frame  # Bottom-right

    # Display the combined output frame
    cv2.imshow("Traffic Detection - 4 Videos", output_frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

for cap in caps:
    cap.release()
cv2.destroyAllWindows()


In [17]:
import cv2
import numpy as np
import time

# Load YOLO with CUDA support (if available)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO names file
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load four videos with your specified file names
caps = [cv2.VideoCapture(f'traffic_vid{i}.mp4') for i in ['', '_2', '_3', '_5']]

# Record start time
start_time = time.time()

while all(cap.isOpened() for cap in caps):
    frames = []
    traffic_counts = []  # To store the number of detected cars for each video
    
    for cap in caps:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    if len(frames) < 4:
        break

    # Check if 10 seconds have passed
    if time.time() - start_time > 10:
        break

    # Resize frames for a 2x2 grid layout (each frame size: 683x384 for a 1366x768 output)
    resized_frames = [cv2.resize(frame, (683, 384)) for frame in frames]

    # Create a black canvas for the output
    output_frame = np.zeros((768, 1366, 3), dtype=np.uint8)

    # Assign each resized frame to its respective quadrant
    output_frame[:384, :683] = resized_frames[0]  # Top-left
    output_frame[:384, 683:] = resized_frames[1]  # Top-right
    output_frame[384:, :683] = resized_frames[2]  # Bottom-left
    output_frame[384:, 683:] = resized_frames[3]  # Bottom-right

    # Process each quadrant independently
    for i, frame in enumerate(resized_frames):
        height, width, channels = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        car_count = 0  # Initialize car count for the current video

        for out in outs:
            for detection in out:
                detection = detection.reshape(-1, 85)
                for obj in detection:
                    scores = obj[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(obj[0] * width)
                        center_y = int(obj[1] * height)
                        w = int(obj[2] * width)
                        h = int(obj[3] * height)

                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)

                        if classes[class_id] == "car":
                            car_count += 1  # Increment the car count if the detected object is a car

        traffic_counts.append(car_count)  # Append the car count for the current video

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)

        if indexes is not None:
            for j in indexes.flatten():
                x, y, w, h = boxes[j]
                label = str(classes[class_ids[j]])
                color = (0, 255, 0) if label == "car" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Place the processed frame back into the output frame
        if i == 0:
            output_frame[:384, :683] = frame  # Top-left
        elif i == 1:
            output_frame[:384, 683:] = frame  # Top-right
        elif i == 2:
            output_frame[384:, :683] = frame  # Bottom-left
        else:
            output_frame[384:, 683:] = frame  # Bottom-right

    # Determine which video has the most traffic
    max_traffic = max(traffic_counts)
    max_traffic_index = traffic_counts.index(max_traffic)

    # Display a message indicating which video has the most traffic
    cv2.putText(output_frame, f"Most Traffic: Video {max_traffic_index + 1}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Display the combined output frame
    cv2.imshow("Traffic Detection - 4 Videos", output_frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

for cap in caps:
    cap.release()
cv2.destroyAllWindows()
